In [5]:
import pandas as pd

# Load the CSV file into a DataFrame
home_prices_df = pd.read_csv("Metro_US_All_Home_Prices.csv")
print(home_prices_df.head())

# Create a new column "Metro City" by extracting the city name from "Region Name"
# We split the string in "Region Name" at the comma and take the first part (city)
home_prices_df["Metro City"] = home_prices_df["RegionName"].str.split(',').str[0]

# Display the updated DataFrame to verify the new "Metro City" column
home_prices_df.head()

   RegionID  SizeRank       RegionName RegionType StateName     2000-01-31  \
0    102001         0    United States    country       NaN  122710.838539   
1    394913         1     New York, NY        msa        NY  218233.216724   
2    753899         2  Los Angeles, CA        msa        CA  222104.914025   
3    394463         3      Chicago, IL        msa        IL  153956.315498   
4    394514         4       Dallas, TX        msa        TX  126111.301852   

      2000-02-29     2000-03-31     2000-04-30     2000-05-31  ...  \
0  122926.726242  123194.458782  123768.692526  124429.566719  ...   
1  219160.584763  220096.541902  221993.153236  223957.159952  ...   
2  222931.426109  224032.332323  226222.765868  228618.413461  ...   
3  154098.834197  154370.263570  155045.512395  155857.755512  ...   
4  126167.472266  126232.045908  126399.736872  126620.832817  ...   

      2023-11-30     2023-12-31     2024-01-31     2024-02-29     2024-03-31  \
0  353806.242676  354242.71700

,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,Metro City
0,102001,0,United States,country,NaN,122710.838539,122926.726242,123194.458782,123768.692526,124429.566719,...,354242.717001,354677.065211,355473.042348,357021.050068,358884.957514,360324.712897,360888.668935,361037.327498,361281.717048,United States
1,394913,1,"New York, NY",msa,NY,218233.216724,219160.584763,220096.541902,221993.153236,223957.159952,...,639265.650987,642178.317205,646255.694516,651704.395720,657863.854229,662561.428749,665667.764258,668437.168324,672210.970325,New York
2,753899,2,"Los Angeles, CA",msa,CA,222104.914025,222931.426109,224032.332323,226222.765868,228618.413461,...,926554.539271,926708.887683,923740.986284,922618.466845,926105.996730,932034.718291,936012.934083,940417.087163,945635.978294,Los Angeles
3,394463,3,"Chicago, IL",msa,IL,153956.315498,154098.834197,154370.263570,155045.512395,155857.755512,...,311913.816691,312578.140750,313972.158826,316395.381263,319279.856054,321409.733463,322638.674660,323378.146509,324304.487366,Chicago
4,394514,4,"Dallas, TX",msa,TX,126111.301852,126167.472266,126232.045908,126399.736872,126620.832817,...,372112.028509,372133.731174,372616.126553,373774.335129,374743.310085,374887.158218,374222.635105,373283.837228,372632.458458,Dallas


In [13]:
commute_df = pd.read_csv("Commute_Time_By_Top_25_Metro_Area_All_Years__Three Buckets_Percentage.csv")
print(commute_df.head())

# Drop rows in the home_prices_df where "Metro City" doesn't match with any city in "Metro" column of commute_df
filtered_home_prices_df = home_prices_df[home_prices_df["Metro City"].isin(commute_df["Metro"])]

# Display the filtered DataFrame to verify the rows have been dropped correctly
filtered_home_prices_df



     Metro    Total  Year  Short Commutes - Less than half-hour  \
0  Atlanta  2232939  2010                              51.70347   
1  Atlanta  2486837  2022                              50.34287   
2  Atlanta  2270521  2021                              52.69746   
3  Atlanta  2715402  2019                              48.09004   
4  Atlanta  2715426  2018                              47.66184   

   Medium Commutes - Half-hour to one hour  Long Commutes - More than one hour  
0                                 36.47753                            11.81900  
1                                 37.80220                            11.85494  
2                                 37.59476                             9.70777  
3                                 37.74237                            14.16759  
4                                 38.01562                            14.32254  


,RegionID,SizeRank,RegionName,RegionType,StateName,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,...,2023-12-31,2024-01-31,2024-02-29,2024-03-31,2024-04-30,2024-05-31,2024-06-30,2024-07-31,2024-08-31,Metro City
1,394913,1,"New York, NY",msa,NY,218233.216724,219160.584763,220096.541902,221993.153236,223957.159952,...,6.392657e+05,6.421783e+05,6.462557e+05,6.517044e+05,6.578639e+05,6.625614e+05,6.656678e+05,6.684372e+05,6.722110e+05,New York
2,753899,2,"Los Angeles, CA",msa,CA,222104.914025,222931.426109,224032.332323,226222.765868,228618.413461,...,9.265545e+05,9.267089e+05,9.237410e+05,9.226185e+05,9.261060e+05,9.320347e+05,9.360129e+05,9.404171e+05,9.456360e+05,Los Angeles
3,394463,3,"Chicago, IL",msa,IL,153956.315498,154098.834197,154370.263570,155045.512395,155857.755512,...,3.119138e+05,3.125781e+05,3.139722e+05,3.163954e+05,3.192799e+05,3.214097e+05,3.226387e+05,3.233781e+05,3.243045e+05,Chicago
4,394514,4,"Dallas, TX",msa,TX,126111.301852,126167.472266,126232.045908,126399.736872,126620.832817,...,3.721120e+05,3.721337e+05,3.726161e+05,3.737743e+05,3.747433e+05,3.748872e+05,3.742226e+05,3.732838e+05,3.726325e+05,Dallas
5,394692,5,"Houston, TX",msa,TX,121737.010654,121759.686522,121674.688937,121725.495460,121772.362850,...,3.048329e+05,3.048268e+05,3.052045e+05,3.061897e+05,3.072086e+05,3.077434e+05,3.076251e+05,3.072616e+05,3.070437e+05,Houston
6,395209,6,"Washington, DC",msa,VA,180842.151942,180989.190625,181260.192650,181873.819130,182806.635695,...,5.501634e+05,5.505870e+05,5.520378e+05,5.550904e+05,5.592981e+05,5.627547e+05,5.644495e+05,5.650157e+05,5.655384e+05,Washington
7,394974,7,"Philadelphia, PA",msa,PA,122513.075912,122785.376616,122971.211007,123429.776479,123873.533424,...,3.530673e+05,3.542810e+05,3.558435e+05,3.581864e+05,3.609425e+05,3.630089e+05,3.639378e+05,3.643231e+05,3.650522e+05,Philadelphia
8,394856,8,"Miami, FL",msa,FL,112841.961497,113151.736319,113479.771936,114142.475317,114783.781240,...,4.780922e+05,4.804784e+05,4.824152e+05,4.845651e+05,4.865473e+05,4.882377e+05,4.888037e+05,4.889947e+05,4.890146e+05,Miami
9,394347,9,"Atlanta, GA",msa,GA,145114.913650,145449.378651,145843.440336,146681.566032,147617.899469,...,3.770676e+05,3.776150e+05,3.785426e+05,3.800731e+05,3.818430e+05,3.829845e+05,3.832073e+05,3.830043e+05,3.828614e+05,Atlanta
10,394404,10,"Boston, MA",msa,MA,216568.526977,217441.540329,218396.691528,220159.489052,222052.850426,...,6.696114e+05,6.715970e+05,6.742857e+05,6.789267e+05,6.849457e+05,6.900598e+05,6.927894e+05,6.943895e+05,6.960745e+05,Boston
